In [11]:
from IPython.display import HTML, display
import nbformat as nbf
import uuid
from google.colab import files
from google.colab import output

# Step 1: Show input UI
html_ui = """
<textarea id="markdownInput" style="width:100%;height:300px;" placeholder="Paste your markdown here..."></textarea><br>
<button onclick="google.colab.kernel.invokeFunction('notebook.convert_markdown_to_ipynb', [document.getElementById('markdownInput').value], {})">
Generate Notebook and Download
</button>
"""
display(HTML(html_ui))

# Step 2: Parse and convert markdown
def convert_markdown_to_ipynb(markdown_text):
    lines = markdown_text.strip().split('\n')
    notebook_cells = []

    inside_code_block = False
    code_lines = []
    markdown_buffer = []
    section_title = None

    def flush_markdown():
        nonlocal section_title
        if markdown_buffer:
            content = '\n'.join(markdown_buffer).strip()
            if content:
                if section_title:
                    html_block = f"""<details open>\n<summary><strong>{section_title}</strong></summary>\n\n{content}\n\n</details>"""
                    notebook_cells.append(nbf.v4.new_markdown_cell(html_block))
                    section_title = None  # reset after use
                else:
                    notebook_cells.append(nbf.v4.new_markdown_cell(content))
            markdown_buffer.clear()

    def flush_code():
        if code_lines:
            content = '\n'.join(code_lines).strip()
            if content:
                notebook_cells.append(nbf.v4.new_code_cell(content))
            code_lines.clear()

    for line in lines:
        if line.strip() == '---':
            flush_markdown()
            flush_code()
            section_title = None
            continue

        if line.strip().startswith('```'):
            if inside_code_block:
                flush_code()
                inside_code_block = False
            else:
                flush_markdown()
                inside_code_block = True
            continue

        if inside_code_block:
            code_lines.append(line)
        else:
            if line.strip().startswith('##'):
                flush_markdown()
                section_title = line.strip('# ').strip()
            else:
                markdown_buffer.append(line)

    flush_markdown()
    flush_code()

    nb = nbf.v4.new_notebook()
    nb['cells'] = notebook_cells
    nb['metadata'] = {
        "kernelspec": {
            "display_name": "Python 3",
            "language": "python",
            "name": "python3"
        },
        "language_info": {
            "name": "python",
            "version": "3.10"
        }
    }

    filename = f"generated_{uuid.uuid4().hex[:6]}.ipynb"
    with open(filename, 'w', encoding='utf-8') as f:
        nbf.write(nb, f)

    files.download(filename)

# Step 3: Register in Colab
output.register_callback('notebook.convert_markdown_to_ipynb', convert_markdown_to_ipynb)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[▶️ Open TXT-to-Notebook Converter in Colab](https://colab.research.google.com/github/drhdt-raipur/ai_for_dld_udemy/blob/main/convert_txt_to_notebook.ipynb)
